In [1]:
import os
os.environ['GROQ_API_KEY'] = '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'

# Phần 1: OpenAI và Gradio

## Bước 1: Cài đặt OpenAI API và Gradio

In [2]:
!pip install -U --quiet groq gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/

## Bước 2: Define các thư viện cần thiết

In [3]:
from groq import Groq
import gradio as gr

client = Groq(api_key=os.environ.get('GROQ_API_KEY'))

## Bước 3: Define function predict và khởi chạy


In [6]:
def predict(message, history):
    history_groq_format = []
    for human, assistant in history:
        history_groq_format.append({"role": "user", "content": human })
        history_groq_format.append({"role": "assistant", "content":assistant})
    history_groq_format.append({"role": "user", "content": message})

    response = client.chat.completions.create(model='llama3-8b-8192',
    messages= history_groq_format,
    temperature=1.0,
    stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(predict).launch(share = True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29d8cb5f429d2c0fae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Phần 2: Giới Thiệu LangChain + Tạo phiên bản số của bạn

In [8]:
!pip install -U --quiet langchain langchain_groq

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
import gradio as gr
from langchain_groq import ChatGroq

In [11]:
chat = ChatGroq(temperature=0)

In [12]:
default = """Bạn là một phiên bản ảo của Trấn Thành. Trấn Thành là một MC nổi tiếng với tính tình hài hước, và được người dân Việt Nam yêu quý.
Một số điều lưu ý về Trấn Thành như sau:
- Trấn Thành ưu tiên sự riêng tư, đừng hỏi những thứ quá cá nhân về Trấn Thành.
- Trấn Thành thích được nói về những bài học cuộc sống cho mọi người học hỏi."""

your_prompt = """YOUR_PROMPT"""

In [17]:
def predict(message, history):
    history_list = []
    history_list.append(SystemMessage(content=default))
    for human, assistant in history:
        history_list.append(HumanMessage(content=human))
        history_list.append(AIMessage(content=assistant))
    history_list.append(HumanMessage(content=message))

    partial_message = ''
    for chunk in chat.stream(history_list):
      if chunk.content is not None:
        partial_message = partial_message + chunk.content
        yield partial_message


gr.ChatInterface(predict).launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://97bd2084b710eba787.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://97bd2084b710eba787.gradio.live
